# Memory Test

This notebook tests the memory class with the airsim car environment and all 3 environments - vector, visual, vector-visual

In [1]:
import sys
import os
from pathlib import Path

%reload_ext autoreload
%autoreload 2

base_path=Path("d:/")
base_path = base_path / "work" / "projects"
for pkg in ['ezai_util','ai_coop_py']:
    pkg_path = base_path / pkg
    pkg_path = str(pkg_path.resolve())
    print(pkg_path)
    if not pkg_path in sys.path:
        sys.path.append(pkg_path)
import ezai_util
import airsim

D:\work\projects\ezai_util
D:\work\projects\ai_coop_py


In [2]:
uenv_path = Path("d:/") / 'work' / 'unity-envs'
uenv_file = uenv_path / "airsim_win" / 'AirSimAssets'
uenv_file = str(uenv_file.resolve())

In [ ]:
memory_capacity = 1000
t_max = 1000

for observation_mode in [0,1,2]:
    print('testing with observation_mode ',observation_mode)
    env = airsim.AirSimEnv(filename=uenv_file, observation_mode = observation_mode)
    try:
        env.info()

        # create a memory for random play
        memory = airsim.Memory(capacity = memory_capacity, 
                        state_shapes = env.observation_space_shapes, 
                        action_shape = env.action_space_shape,
                        seed = 123)
        memory.info()

        # do the random play and fill the memory

        t=0
        episode_done = False

        #1. observe initial state
        s = env.reset()

        while not episode_done:
            #2. select an action, and observe the next state
            a = env.action_space.sample()
            s_, r, episode_done, info = env.step(a)

            if(t == t_max-1):
                episode_done=True

            #3. save in memory

            #s = np.asarray(s).squeeze()
            #a = np.asarray(a).squeeze()
            #r = np.asarray(r).squeeze()
            #s_ = None if episode_done else s_ #np.asarray(s_).squeeze()
            memory.append(s=s,a=a,r=r,s_=s_, d=episode_done)

            # update the counters
            t += 1

            if (t_max and t >= t_max):
                break

        # sample the memory
        s,a,r,s_,d = memory.sample(100)
        airsim.Memory.sample_info(s,a,r,s_,d)
    finally:
        env.close()

testing with observation_mode  0


In [ ]:
env.close()